In [1]:
import boto3
client = boto3.client('batch')
def submit_aws_job(*args, local=False, **kwargs):
    if not local:
        return client.submit_job(*args, **kwargs)

In [2]:
def submit_match(a, b, pool, side, timeout=600 , local=False):
    return submit_aws_job(
        jobDefinition='run-match',
        jobName=pool,
        jobQueue='matches',
        containerOverrides={
            'command': ["python", "simulate.py"],
            'environment': [
                {'name': 'PLAYER_A', 'value': a % 0},
                {'name': 'PLAYER_B', 'value': b % 1},
                {'name': 'POOL','value': pool},
                {'name': 'SIDE', 'value': str(side)},
            ]
        },
        timeout={'attemptDurationSeconds': timeout},
        local=local,
    )

In [3]:
def sumbit_symmetric_match(a, b, pool, timeout=600, local=False):
    submit_match(a, b, pool, side=0, timeout=timeout, local=local)
    #submit_match(b, a, pool, side=1, timeout=timeout, local=local)

In [10]:
fine_search_space = [0.5, 0.7, 0.9]
for i in range(25):
    for c in fine_search_space:
            player = "UCTPlayer(%s, td(seconds=5), c=math.sqrt(2)/2)"
            opponent = f"UCTPlayer(%s, td(seconds=5), c={c:.2f})"

            sumbit_symmetric_match(player, opponent, "uct-tuning-c-fine")

In [9]:
fine_search_space = [0.95]
for i in range(25):
    for c in fine_search_space:
            player = "GreedyUCTPlayer(%s, td(seconds=5), c=math.sqrt(2)/2)"
            opponent = f"GreedyUCTPlayer(%s, td(seconds=5), c={c:.2f})"

            sumbit_symmetric_match(player, opponent, "greedy-uct-tuning-c-fine")